# Aula 6 - otimização bayesiana

Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) Introdução
- 2) Otimização bayesiana
- 3) Introdução a LIME e SHAP

Atenção! Precisaremos de uma instalação: 

[hyperopt](http://hyperopt.github.io/hyperopt/)

Então faça o: `pip install hyperopt`

In [1]:
#pip install hyperopt

## 1) Introdução

Desde que conhecemos os primeiros estimadores utilizados para a construção de modelos, uma preocupação adicional nos acompanhou: o ajuste de valores dos hiperparâmetros.

Quase todos os estimadores que conhecemos têm um ou mais **hiperparâmetros** associados.

Como vimos, os hiperparâmetros influenciam o comportamento do modelo (e, portanto, são muito importantes), mas eles não são determinados a partir dos dados! É nosso dever, como cientistas de dados, fornecer valores adequados para os hiperparâmetros.

No começo de nossa jornada, nós utilizávamos os valores default para os hiperparâmetros, ou então, mudávamos manualmente alguns valores, para sensibilizar a influência dos hiperparâmetros no modelo final.

Mas logo percebemos que essa não era a melhor abordagem --- precisávamos de um método mais sistemàtico para a busca de bons valores para os hiperparâmetros!

_______

Para este fim, introduzimos inicialmente o **grid search**, que **testa exaustivamente** combinações explícitas de valores de hiperparâmetros.

Apesar de simples e direta, esta é uma abordagem computacionalmente muito custosa (sentimos isso na pele, ao rodar rotinas de grid search que demoram horas, ou até mesmo dias!). 

De fato, esta é uma abordagem de força bruta que, aliás, não nos dá garantia nenhuma: é perfeitamente possível que nenhuma das combinações que estabelecemos seja boa boa! Oras, há muitos casos em que o **espaço de hiperparâmetros** é infinito (basta considerar um hiperparâmetro dado por um float que não é limitado).

______

Para endereçar este problema, introduzimos então o **random search**.

Neste caso, ao invés de disponibilizarmos valores específicos de hiperparâmetros a serem combinados, disponibilizamos **intervalos** ou explicitamente **distribuições de probabilidade** sobre o espaço de parâmetros de cada hiperparâmetro, e **amostramos valores aleatórios** destas distribuições para gerar as combinações.

Com essa abordagem estocástica, há a possibilidade de encontrarmos boas combinações, que não estariam inclusas nos valores explícitos que passamos para o grid search.

Por outro lado, com o random search também não temos garantia alguma que as melhores combinações serão encontradas, sobretudo porque **todas as combinações são amostradas de maneira aleatória**. Ou seja, todas as combinações são amostradas **sem considerar** a performance das combinações anteriores. Com esta metodologia, fica difícil garantirmos que efetivamente encontraremos boas combinações...

Metodologia de grid search (à esquerda) vs random search (à direita):

<img src=https://www.researchgate.net/profile/Minrui-Zheng/publication/328252103/figure/fig4/AS:766093471801344@1559662325592/Distribution-of-sampled-hyperparameters-a-grid-search-b-random-search.png width=600>

_________

Então, podemos nos perguntar: qual seria o próximo passo? Existiria um método ainda mais eficiente e "educado" para fazer a otimização de hiperparâmetros?

A resposta é: sim! Hoje, conheceremos o método de **otimização Bayesiana**, aplicada à otimização de hiperparâmetros!

_________
_________
_________

## 2) Otimização Bayesiana

O objetivo da otimização baeysiana é o mesmo que tínhamos com o grid e random search: **determinar os valores adequados para um conjunto de hiperparâmetros**, de modo que a combinação de valores **proporcione os melhores valores para uma métrica de avaliação calculada em dados de validação**.

O que muda com a otimização baeysiana é a forma como isso será feito.

Já descrevemos os problemas que ambos grid e random search têm. Sobretudo, o fato das combinações de valores dos hiperparâmetros serem independentes entre si, sem levar em consideração combinações anteriores, que poderiam ser boas.

>Pra entender isso melhor, imagine o cenário em que queremos otimizar 3 hiperparâmetros.
<br><br>
>Pode ser que, em uma combinação, encontremos bons valores para 2 dos 3 hiperparâmetros; ja, na próxima, como os 3 valores serão novos, pode ser que os 3 sejam ruins. Não seria interessante de conseguíssemos manter a informação sobre os valores bons que encontramos?

Esse é o espírito da otimização baeysiana!

Pra entender isso intuitivamente, considere a figura a seguir (suponha que o "score" é um erro, então, quanto menor, melhor):

<img src=https://miro.medium.com/max/1120/1*MiNXGrkk5BbjfkNAXZQSNA.png width=400>

Pergunta: onde você concentraria a busca por valores do hiperparâmetro `n_estimators`?

Olhando pro gráfico, existe claramente uma região no espaço do hiperparâmetro `n_estimators` que é melhor: menos de 200 árvores.

**Uma vez que temos este conhecimento**, realmente não faz sentido algum que os próximos valores que vamos testar sejam maiores que 200, não é mesmo?

Esse é o espírito da otimização bayesiana: **utilizamos a informação que temos quanto ao score alcançado com valores iniciais de hiperparâmetros para guiar a escolha dos próximos valores!**

E aqui já fica claro o porquê do método receber o título "bayesiano": de fato, estamos nos utilizando da "filosofia" bayesiana de ajuste de estratégia conforme agregamos novas informações! 

>E é por isso que a otimização bayesiana é tão mais eficiente que as estratégias anteriores: gastamos um pouco mais de energia para propor combinações de hiperparâmetros **levando em consideração** as combinações passadas e suas respectivas perfomances.
<br><br>
Com isso, podemos focar apenas em testar **combinações promissoras**, e não precisamos gastar energia em procurar por combinações que sabemos não ser tão boas.

Faz sentido, não é mesmo?

Agora, precisamos apenas entender **como** é possível implementar isso.

### 2.1) Função objetivo

É razoável assumirmos que existe uma função que relaciona a métrica de performance (score) que temos interesse de otimizar e os hiperparâmetros, não é mesmo?

Sabendo que as métricas de performance essencialmente comparam os targets preditos com os targets reais, e sabendo que os targets preditos (pela hipótese) são influenciados pelos hiperparâmetros, é bem natural que assumamos que esta função existe.

Agora, o ponto é que pode ser extremamente difícil escrever esta função explicitamente. Como nosso objetivo será encontrar valores que **otimizam** a função, costumamos chamá-la de **função objetivo**. 

Mas, veja, queremos otimizar uma função objetivo **sem nem saber qual é sua dependência funcional com os hiperparâmetros**. Problemas deste tipo são chamados de **otimização de caixa-preta**, e existem várias técnicas para abordá-los. Recomendo [este material](https://www.lix.polytechnique.fr/~dambrosio/blackbox_material/Cassioli_1.pdf) aos interessados.

<img src=https://www.researchgate.net/profile/Abraham-Duarte/publication/236164556/figure/fig1/AS:393408515461120@1470807307227/Schematic-representation-of-the-black-box-optimization-framework.png width=500>

Quando formos pra prática, construiremos a função objetivo implicitamente, como o resultado retornado por uma **métrica de avaliação** dado o treinamento de um modelo. 

Na prática, isso nos será suficiente, pois bastará que tomemos **alguns pontos** desta função, para seguir com sua otimização, isso graças à introdução do surrogate model.

### 2.2) Surrogate model

Na prática, iremos apenas **avaliar alguns pontos da função objetivo** e construir um **modelo probabilístico dela** com base nestes pontos, que serão tratados como **amostras**.

Este modelo probabílistico é conhecido como **surrogate model (modelo substituto ou emulador)**, pois ele emula probabilisticamente o que seria a função real, sem a necessidade dela ser reconstruída explicitamente. E o melhor, dado que o modelo surrogate é bem mais simples que a função objetivo, será muito mais fácil otimizá-lo do que a função objetivo em si!

<img src=https://www.researchgate.net/profile/Seung-Seop-Jin-2/publication/297605027/figure/fig1/AS:339392045568000@1457928777016/Concept-of-sequential-surrogate-modelling.png width=600>

O ponto é que nosso modelo surrogate será atualizado **de maneira bayesiana**, isto é, levando em consideração scores anteriores, para sugerir as próximas combinações de hiperparâmetros de maneira mais principada.

Operacionalmente, seguiremos o seguinte passo-a-passo:

- Construiremos um modelo probabilístico surrogate para a função objetivo;
- Otimizaremos o modelo surrogate, encontrando bons valores para os hiperparâmetros nesta função;
- Utilizamos estes valores encontrados e damos de input para a função objetivo real, e amostramos um novo ponto;
- Atualizamos o modelo surrogate, incorporando o novo ponto amostrado;
- Repetimos os últimos 3 passos, até atingirmos o critério de parada (em geral, um número pré-definido de passos).


Há muitas formas possíveis para o modelo surrogate. Um método muito interessante é o que utiliza processos gaussianos. Sugiro muito a leitura [deste post interativo](https://distill.pub/2019/visual-exploration-gaussian-processes/) para conhecer um pouco mais sobre este método; e [este post](https://towardsdatascience.com/the-intuitions-behind-bayesian-optimization-with-gaussian-processes-7e00fcc898a0) para uma intuição quanto a sua aplicação como modelo surrogate da otimização bayesiana.

Na nossa implementação prática, utilizaremos o método conhecido como **Tree-structured Parzen Estimator** (TPE) para a construção do modelo surrogate.

Não nos preocuparemos com a matemática por trás do método. (Aos interessados, sugiro a leitura [deste artigo](https://www.diva-portal.org/smash/get/diva2:1223709/FULLTEXT01.pdf), que compara diferentes modelos surrogates e técnicas de otimização de hiperparâmetros, no contexto de redes neurais, embora as ideias sejam válidas no geral).

Basta sabermos o seguinte: o TPE precisa de um **threshold de score**:

<img src=https://miro.medium.com/max/1120/1*H5pyf3G115WGJwPpg65yaQ.png width=400>

Com base neste score, o método gera distribuições de probabilidade a priori, para ambos os valores de hiperparâmetros que estão acima e abaixo do threshold.

Uma vez em posse destas distribuições, o método se utiliza então de um **critério de escolha** que favorece valores dos hiperparâmetros que estão do lado do threshold que nos interessa (no caso deste exemplo, abaixo do threshold), e estes valores são propostos como os próximos hiperparâmetros a serem escolhidos.

E é assim que o TPE consegue "focar nas regiões promissoras" do espaço de hiperparâmetros. Muito legal, não é mesmo?

Mas ainda resta uma pergunta: qual é exatamente este critério de escolha das regiões? **Como**, a cada iteração, os próximos valores de hiperparâmetros são propostos, com o objetivo de minimizar a função objetivo?

Isto é feito de acordo com a chamada **função de seleção**.

### 2.3) Função de seleção

O objetivo da função de seleção é muito simples: **determinar os valores dos hiperparâmetros que são escolhidos do modelo surrogate** a cada passo.

Há vários critérios possíveis, mas o mais comum é conhecido como **Expected Improvement** (melhoria esperada), e ela é uma função do modelo surrogate. Seu uso é bem direto: **propomos novos valores de hiperparâmetros de modo que o expected improvement seja maximizado**. 

Podemos omitir aqui os detalhes matemáticos, o importante é lembrarmos que é o papel da função de seleção estabelecer o critério segundo o qual o modelo surrogate irá proporcionar os valores de hiperparâmetros a cada iteração.

E, com isso, conseguimos, a cada iteração, propor candidatos a valores de hiperparâmetros que melhoram o modelo surrogate. O registro destes valores é muito importante, para que o teorema de Bayes entre em ação!

### 2.4) Histórico de registros

O histórico de registros, que é uma tupla com **(valores de hiperparâmetros, respectivo score)** é muitíssimo importante para que seja possível a construção de um modelo surrogate que, a cada passo, melhora a descrição probabilística da função objetivo.

Isso é importante pois, quão melhor for nossa descrição da função objetivo, teremos que o passo disponibilizado pelo expected improvement irá proporcionar hiperparâmetros mais próximos do ótimo da função objetivo. 

E é aqui que o teorema de Bayes entra em ação: a partir de novas evidências (amostras do surrogate), atualizamos a nossa descrição da funcção objetivo com cada vez mais detalhes, e, assim, ela fica mais fácil de ser otimizada, sem precisar ser explicitamente construída!

<img src=https://miro.medium.com/max/1400/1*RQ-pAwQ88yC904QppChGPQ.png width=500>

<img src=https://miro.medium.com/max/1400/1*bSLAe1LCj3mMKfaZsQWCrw.png width=500>

Agora que já conhecemos todos os ingredientes importantes, podemos dizer enunciar o passo-a-passo da otimização bayesiana:

- Primeiro definimos um espaço de busca para cada um dos hiperparâmetros, impondo uma distribuição de probabilidade sobre o espaço;
- Construímos implicitamente a função objetivo, que tem como inputs os hiperparâmetros, e como output o score a ser otimizado (que será a métrica de avaliação de interesse);
- Construímos o modelo surrogate probabilístico da função objetivo;
- Construímos a função de seleção, que irá determinar os próximos valores para os hiperparâmetros a serem escolhidos do modelo surrogate;
- Atualizamos um histórico de valores de hiperparâmetros e respectivo score, que é utilizado pelo algoritmo para a atualização do modelo surrogate;
- Repetimos o processo até o critério de parada.


### Em resumo, a otimização Bayesiana funciona da seguinte forma:

Para cada hyperparâmetro temos uma função Surrogate que calcula o score para esse parâmetro. Essencialmente aqui, está sedo feito um ajuste de curva da função surrogate com os parâmetros e scores.

Porém, essa curva não é ajustada em todos os pontos, de acordo com amostras e com o comportamento dos scores, ela define atrvés do TPE regiões que vale mais a pena realizar um ajuste mais preciso.

Definindo essas "Regiões de confiança", a função foca em testar mais combinações associadas a essa região e abandona combinações de parâmetros distântes dessa região.

O foco então é maximizar a Esperança (Expected Improvement) de todas as funções Surrogates. Como se fosse uma "média" de todos melhores scores obtidos pelas funções surrogates.

Maximizando o Expected Improvement, obtemos os melhores scores com as funções surrogate. Estes melhores scores são obtidos nos melhores hiperparâmetros combinados. 
Ou seja, consequentemente, conseguimos obter os melhores parametros para uma função de Machine Learning otimizando uma função muito mais simples.


__________

E é isso!

Para os interessados em maiores detalhes matemáticos, recomendo fortemente [este artigo](http://proceedings.mlr.press/v28/bergstra13.pdf), e [este também](https://proceedings.neurips.cc/paper/2011/file/86e8f7ab32cfd12577bc2619bc635690-Paper.pdf), que reporta resultados incríveis a favor do uso de otimização bayesiana como alternativa aos métodos tradicionais de otimização de hiperparâmetros.

Agora, vamos ver o método funcionando na prática!

______________

In [2]:
# Começar com as importações iniciais
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

### 1. Vamos inicial com uma aplicação bastante simplória

In [3]:
# importamos o dataset

In [4]:
df = pd.read_csv("heart.csv")
df.shape

(303, 14)

In [5]:
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [6]:
# Separamos x e y

In [7]:
X = df.select_dtypes(include = np.number).drop(columns = ["output"])
y = df["output"]

In [8]:
# fazemos o split dos dados

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

In [11]:
# Vamos fazer uma otimização comum para o paralelo

In [12]:
# Como foi nos dados de treino? 

In [13]:
# Como foi nos dados de teste? 

### Vamos realizar agora o processo de OTIMIZAÇÃO BAYESIANA

In [14]:
# inicialmente fazemos os importes para isso

In [15]:
# Agora, assim como temos o dicionário "params_grid", aqui criamos o "hps_space"

In [16]:
# Agora, precisamos criar nossa FUNÇÃO OBJETIVO:
#    Ela recebe o "hps_space" como parâmetro e deve retornar que queremos MÁXIMIZAR

# REPITO, MAXIMIZAR!

Agora, instanciamos a classe [Trials()](http://hyperopt.github.io/hyperopt/getting-started/minimizing_functions/#the-trials-object), que é a interface que nos permitirá realizar a otimização bayesiana, e que guarada o histórico, como veremos.

Note que também passamos o TPE como algoritmo para o modelo surrogate, o que é dado pelo `tpe.suggest`.

Indicamos também o número máximo de passos que queremos, com o `max_evals`.

A otimização em si é realizada com a chamada da função `fmin()`.

In [17]:
# pré-resposta da otimização

In [18]:
# a resposta final, vem com o space_eval

In [19]:
# Agora podemos definir nosso modelo otimizado!

In [20]:
# Como foi nos dados de treino? 

In [21]:
# Como foi nos dados de teste? 

### Vale testar os tempos de processamento 

In [22]:
# %%timeit
# random_gb.fit(X_train, y_train)

# 884 ms ± 44.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [23]:
# %%timeit  

# trials = Trials()

# best_hps = fmin(funcao_objetivo, # função objetivo
#                 space=hps_space, # o espaço de parâmetros
#                 algo=tpe.suggest, # o tpe para o surrogate
#                 trials=trials, # trials para registrar o histórico
#                 rstate=np.random.default_rng(42), # semente randomica 
#                 max_evals=10) # número máximo de avaliações
# best_hps = space_eval(hps_space, best_hps)

#1.1 s ± 21.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

### E se eu quisesse entender o que meu modelo está fazendo?

#### Para 1 dado em específico: LIME

In [24]:
# instale o lime
# importe o lime_tabular

In [25]:
# Agora, precisamos de coisas do modelo:

In [26]:
# resgatamos o modelo

In [27]:
# resgatamos os nomes dos features 

In [28]:
# resgatamos as classes que ele treinou 

In [29]:
# Agora "treinamos" o lime_tabular 

In [30]:
# Agora, escolha 1 dado específico

In [31]:
# Agpra pedimos a explicacao de um dado em específico

In [32]:
# E podemos observar essa explicação

In [33]:
# Até mesmo como forma de lista

#### E se eu quiser saber como as variáveis estão influenciando no modelo em geral? SHAP

In [34]:
# intale o shap
# importe o shap

In [35]:
# Passamos o modelo para o expolicador do shap

In [36]:
# resgatamos os valores shap a partir dos dados de treino

In [37]:
# Podemos visualizar a importância de diversas formas 

### Agora vamos tentar com um pipeline completo

In [38]:
df.head(5)

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

In [40]:
# Organizamos o pipeline para features numéricos

In [41]:
# Salvamos os nomes para depois
features_num = X_train.select_dtypes(include=np.number).columns.tolist()

In [42]:
# Organizamos o pipeline para features categoricos

In [43]:
# Também salvamos os nomes para depois
features_cat = X_train.select_dtypes(exclude=np.number).columns.tolist()

In [44]:
# Definimos o pipeline que processará por completo

In [45]:
# Agora, se vamos implementar a OTMIZAÇÃO BAYESIANA:

In [46]:
# Denifimos o espaço de parâmetros

In [47]:
# Definimos a função objetivo

In [48]:
# Organizamos como essa função deverá ser minimizada

In [49]:
# Resposta inicial antes de otimizar

In [50]:
# resposta final considerando nossos parâmetros

In [51]:
# Instanciamos o modelo com os novos parâmetros

In [52]:
# Como foi nos dados de treino? 

In [53]:
# Como foi nos dados de teste? 

## Exercício 1:

Crie um modelo para otmização Bayesiana assim como foi feito, porém utilize o dataset "german_credit_risk" e OUTRO MODELO:

    - LogisticRegression;
    - DecisioTreeClassifier;
    - XGBoostClassifier;

## Exercício 2: 

Refaça a implementação realizada em aula, porém utilizando a biblioteca [scikit-optimize](https://towardsdatascience.com/hyperparameter-optimization-with-scikit-learn-scikit-opt-and-keras-f13367f3e796)

(O link não é da documentação, mas sim um exemplo de implementação)